# Flight Feature Engineering

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@231ae096
session = org.apache.spark.sql.SparkSession@231ae096
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovya...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovya...

# Loading Data

In [4]:
import com.flightdelay.data.loaders.FlightDataLoader
FlightDataLoader.loadFromConfiguration()
val rawFlightPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"

val flightsDF = spark.read.parquet(rawFlightPath)


[STEP 1][DataLoader] Flight Data Loading - Start

Loading from CSV file:
  - Path: /home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv
  - Loaded 18286055 records from CSV

Saving to Parquet format:
  - Path: /home/jovyan/work/output/common/data/raw_flights.parquet
  - Saved 18286055 records to Parquet

Schema:
root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



rawFlightPath = /home/jovyan/work/output/common/data/raw_flights.parquet
flightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

In [13]:
import org.apache.spark.sql.functions._

val filteredFlightsDF = flightsDF.filter(
  col("ARR_DELAY_NEW") >= 15 && 
  (col("ARR_DELAY_NEW") =!= (coalesce(col("WEATHER_DELAY"), lit(0)) + coalesce(col("NAS_DELAY"), lit(0))) ) &&
  (col("WEATHER_DELAY")===0 || col("NAS_DELAY")===0)
)

filteredFlightsDF.count()

filteredFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


2757876

In [15]:
import com.flightdelay.data.preprocessing.flights.FlightDataSetFilterGenerator

val flightsWithDxDF = FlightDataSetFilterGenerator.preprocess(flightsDF)
flightsWithDxDF.printSchema

[FlightDataSetFilterGenerator] Total vols: 18286055
 - D1     :   710294 vols (3.88%)
 - D3     :  2024941 vols (11.07%)
 - D4     :  7017890 vols (38.38%)
 - D2_15  :    53900 vols (0.29%)
 - D2_30  :    26658 vols (0.15%)
 - D2_45  :    14062 vols (0.08%)
 - D2_60  :     7797 vols (0.04%)
 - D2_90  :     2570 vols (0.01%)
root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- D4: integer (nullable = false)
 |-- D3: integer (nullable = false)
 |-- D1: integer (nullable = false)
 |-- D2_15: int

flightsWithDxDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 18 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 18 more fields]

In [16]:
// D2 Recalculs

import org.apache.spark.sql.functions._

val filteredFlightsDF = flightsDF.filter(
  col("ARR_DELAY_NEW") >= 15 && 
  (col("ARR_DELAY_NEW") === (coalesce(col("WEATHER_DELAY"), lit(0)) + coalesce(col("NAS_DELAY"), lit(0))) ) &&
  (col("NAS_DELAY")>= 15)
)

filteredFlightsDF.count()

filteredFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


653895

In [17]:
// D2 Recalculs

import org.apache.spark.sql.functions._

val filteredFlightsDF = flightsDF.filter(
  col("ARR_DELAY_NEW") >= 15
)

filteredFlightsDF.count()

filteredFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


3524963